In [14]:
# Settings

# General settings
data_folder = "/home/andrea/CIMA/PROJECT_Volta/data/integrated_data/" #"/home/andrea/Desktop/Test/series/" 
start_analysis = "2018-01-01 00:00"
end_analysis = "2020-12-30 00:00"
freq="D"

# Hmc settings
domain = "volta"
hmc_output = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/hmc.hydrograph.txt"
hmc_output_old = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/hmc.hydrograph.txt_gd_ok"
hmc_static_gridded = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/data_static/gridded/"
hmc_static_point = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/data_static/point/"

# Data
et_series = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/average_ETCum.txt"
et_pot_series = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/average_ETPotCum.txt" 
rainfall_series = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/average_rainfall_mm_h.txt"

In [48]:
# Imports
%matplotlib inline

from ipywidgets import interactive,HBox
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pysheds.grid import Grid

import ipywidgets as widgets
from IPython.display import display, HTML
import os, glob
import datetime as dt
import matplotlib.lines as mlines

# Define custom functions
def create_df(choices):
    df = pd.DataFrame(index=pd.date_range(start_time,end_time,freq=freq), columns=choices)
    for name in choices:
        series = pd.read_csv(os.path.join(data_folder, name + ".csv"), index_col=0, header=0, parse_dates=True)
        df[name] = series.reindex(pd.date_range(start_time,end_time,freq=freq), method=None)
    return df

def multiplot(widg):
    choices = widg['new']
    df = create_df(choices)
    data = df.loc[:, choices] if choices else df
    output.clear_output(wait=True)
    with output:
        ax = data.plot(figsize=(10,7))
        plt.show()
        
def combinedplot(widg):
    choices = widg['new']
    data_etPot = series["ETPot"].loc[:, choices] if choices else series
    data_et = series["ET"].loc[:, choices] if choices else series
    data_rain = series["rain"].loc[:, choices] if choices else series
    data_qMod = series["discharge_mod"].loc[:, choices] if choices else series
    data_qModOld = series["discharge_mod_old"].loc[:, choices] if choices else series
    data_qObs = series["discharge_obs"].loc[:, choices] if choices else series
    output2.clear_output(wait=True)
    with output2:
        ax = data_rain.plot(figsize=(15,7), color='c') 
        plt.ylim(bottom=0)
        plt.ylabel("mm of rain")
        axx = ax.twinx()
        ax1 = data_et.plot(figsize=(15,7), color='g')
        ax4 = data_etPot.plot(figsize=(15,7), color='y')
        ax2 = data_qMod.plot(figsize=(15,7), color='b')
        ax5 = data_qModOld.plot(figsize=(15,7), color='b', style='--')
        ax3 = data_qObs.plot(figsize=(15,7), color='r', style='.')
        plt.ylim(bottom=0)
        plt.ylabel("mm")
        cyan_line = mlines.Line2D([], [], color='cyan', label='rain')
        green_line = mlines.Line2D([], [], color='green', label='et')
        yellow_line = mlines.Line2D([], [], color='yellow', label='et pot')
        blue_line = mlines.Line2D([], [], color='blue', label='mod_dis')
        blue_dot_line = mlines.Line2D([], [], color='blue', linestyle='--', label='mod_dis_old')
        red_line = mlines.Line2D([], [], color='red', linestyle='', marker='.', label='obs_dis')
        
        plt.legend(handles=[cyan_line, green_line, yellow_line, blue_dot_line, blue_line, red_line])
        plt.show()

def read_discharge_hmc(file='', col_names=None):
    custom_date_parser = lambda x: dt.datetime.strptime(x, "%Y%m%d%H%M")
    hmc_discharge_df = pd.read_csv(file, header=None, delimiter=r"\s+", parse_dates=[0], index_col=[0], date_parser=custom_date_parser)
    hmc_discharge_df.columns = col_names
    return hmc_discharge_df

In [34]:
# Read static inputs

# Sections
sections = tabular = pd.read_csv(os.path.join(hmc_static_point, domain + ".info_section.txt"), sep="\s+", header=None)
rHMC, cHMC, basin_name, section_name = tabular.values[:,0], tabular.values[:,1], tabular.values[:,2], tabular.values[:,3]

# Pointers
grid = Grid.from_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"))
pnt = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"), dtype=np.int8)
areacell = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".areacell.txt"))

In [35]:
# Compute basin cell area
basin_area = pd.DataFrame(index=section_name, columns=["Area (m2)"])
dirmap_HMC = (8, 9, 6, 3, 2, 1, 4, 7)

for ix, iy, basin, name in zip(cHMC, rHMC, basin_name, section_name):
        basin = grid.catchment(fdir=pnt, x=ix-1, y=iy-1, dirmap=dirmap_HMC, xytype='index')
        mask = np.where(basin>0, 1, np.nan)
        basin_area.loc[name, "Area (m2)"] = np.nansum(mask*areacell).astype("float32")

In [36]:
# Read hmc output
mod_out = read_discharge_hmc(hmc_output, section_name)
mod_out_old = read_discharge_hmc(hmc_output_old, section_name)   ################

In [37]:
# Set timing
start_time = dt.datetime.strptime(start_analysis, "%Y-%m-%d %H:%M")
end_time = dt.datetime.strptime(end_analysis, "%Y-%m-%d %H:%M")

In [38]:
# Read datasets
series = {}

series["ET"] = pd.read_csv(et_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
series["ETPot"] = pd.read_csv(et_pot_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
series["rain"] = pd.read_csv(rainfall_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
dis_out = mod_out[start_time:end_time]
dis_out_old = mod_out_old[start_time:end_time]  ########################àà

missing_data = [i for i in section_name if not os.path.isfile(os.path.join(data_folder, i + ".csv"))]
display("WARNING! Data for sections " + ", ".join(missing_data) + " are missing!")
dis_in = create_df([i for i in section_name if i not in missing_data])

'WARNING! Data for sections Bayé, Gorée, Pletou, Rte-SokodeBassar are missing!'

In [39]:
# Convert all series to mm/day
series["rain"] = series["rain"].resample("D").sum()
series["discharge_mod_old"] = pd.DataFrame(index = series["rain"].index, columns = dis_out_old.columns) ########
series["discharge_mod"] = pd.DataFrame(index = series["rain"].index, columns = dis_out.columns)
series["discharge_obs"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)
series["availability"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)

for name in dis_out.columns:
    series["discharge_mod"][name] = (dis_out[name] * (1000*3600)/float(basin_area.loc[name, "Area (m2)"])).resample("D").sum()
    series["discharge_mod_old"][name] = (dis_out_old[name] * (1000*3600)/float(basin_area.loc[name, "Area (m2)"])).resample("D").sum()
    ########
    
for name in series["discharge_obs"].columns:
    series["discharge_obs"][name] = (dis_in[name] * (1000*3600*24)/float(basin_area.loc[name, "Area (m2)"]))
    series["availability"][name] = dis_in[name].resample("D").count()

In [40]:
series_annual = {}

for stype in series.keys():
    series_annual[stype] = series[stype].resample("Y").sum()
    index = pd.Index([str(y) + "_" + stype for y in np.unique(series_annual[stype].index.year)])
    series_annual[stype].set_index(index, inplace=True)

In [41]:
out_table = pd.concat([series_annual[i].T for i in series_annual.keys()], axis=1)
out_table = out_table.reindex(sorted(out_table.columns), axis=1)

display(out_table)
#out_table.to_csv("/home/andrea/Desktop/summary_balance_flip_ok.csv")

,2018_ET,2018_ETPot,2018_availability,2018_discharge_mod,2018_discharge_mod_old,2018_discharge_obs,2018_rain,2019_ET,2019_ETPot,2019_availability,...,2019_discharge_mod_old,2019_discharge_obs,2019_rain,2020_ET,2020_ETPot,2020_availability,2020_discharge_mod,2020_discharge_mod_old,2020_discharge_obs,2020_rain
Arly_doudouro,783.550672,2580.409468,0.0,76.567980,268.301827,0.000000,900.261952,868.925443,2512.963376,0.0,...,362.852382,0.000000,1051.774522,813.642008,2589.130699,0.0,205.061792,389.351040,0.000000,1016.177059
Atchangbade,789.861612,1486.747025,0.0,153.606145,265.482792,0.000000,1024.472672,760.240346,1527.292698,0.0,...,301.155954,0.000000,1053.661974,845.617690,1565.102488,0.0,339.322508,458.327787,0.000000,1287.247112
Badara,859.788384,2438.570312,290.0,253.908204,455.019691,79.559732,1194.545892,995.828217,2361.585523,299.0,...,548.811159,49.561959,1357.211007,879.839296,2436.950789,312.0,187.439285,379.112180,66.444979,1061.566361
Bagre_aval_au_pont,749.079553,2701.775979,287.0,84.647796,302.036521,2.948007,904.982269,782.967575,2731.391996,365.0,...,245.128790,0.138518,869.185845,731.028527,2769.938888,257.0,153.910860,323.414562,2.828404,886.024782
Bamboi,806.437338,2396.379863,365.0,139.051647,336.654935,73.350016,1009.589251,871.028425,2372.538584,365.0,...,346.777570,89.788584,1037.507046,828.298968,2415.029783,0.0,143.292765,321.666186,0.000000,959.176227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yilou,774.933880,2777.467558,0.0,99.072151,304.198391,0.000000,926.986042,772.517121,2855.452611,0.0,...,198.992715,0.000000,801.137077,737.991786,2828.094165,0.0,111.869565,286.281721,0.000000,852.425244
Ziou,773.729611,2582.931195,359.0,94.294374,305.413891,56.807634,921.873239,877.396268,2532.097739,363.0,...,434.600233,44.856950,1133.480812,762.083111,2668.340677,274.0,197.210856,369.959522,49.546870,946.535514
Gbanlou-bineda_Kopingue,NaN,NaN,86.0,33.097490,199.990767,45.378986,NaN,NaN,NaN,57.0,...,241.970299,36.949647,NaN,NaN,NaN,70.0,48.648515,198.764493,44.664263,NaN
Kara-kpessidè,NaN,NaN,0.0,239.225315,388.543529,0.000000,NaN,NaN,NaN,0.0,...,417.571551,0.000000,NaN,NaN,NaN,0.0,419.643209,570.159121,0.000000,NaN


In [49]:
# Generate list
selector2 = widgets.Dropdown(
options=series_annual["ET"].T.index,
value=series_annual["ET"].T.index[0])

output2 = widgets.Output()

# Set layout and display
form_item_layout = widgets.Layout(
    display='flex',
    justify_content='space-between'
)
display(widgets.VBox([selector2, output2], layout=form_item_layout))

# Re-generate multiplot
selector2.observe(combinedplot, names='value')